In [2]:
import os
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms.bedrock import Bedrock
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import BedrockEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [3]:
import json
import os
import sys
import boto3
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww
# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
os.environ["AWS_DEFAULT_REGION"] = "us-west-2" # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::195364414018:role/Crossaccountbedrock"
bedrock_runtime = bedrock.get_bedrock_client(
assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-west-2
  Using role: arn:aws:iam::195364414018:role/Crossaccountbedrock ... successful!
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [4]:
def OLD_get_llm():
    model_kwargs = { #AI21
    "maxTokens": 1024,
    "temperature": 0,
    "topP": 0.5,
    "stopSequences": ["Human:"],
    "countPenalty": {"scale": 0 },
    "presencePenalty": {"scale": 0 },
    "frequencyPenalty": {"scale": 0 }
    }
    llm = Bedrock(
    credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"), #sets the profile name to 
    region_name=os.environ.get("BWB_REGION_NAME"), #sets the region name (if not the defaul
    endpoint_url=os.environ.get("BWB_ENDPOINT_URL"), #sets the endpoint URL (if necessary)
    model_id="ai21.j2-ultra-v1", #set the foundation model
    model_kwargs=model_kwargs) #configure the properties for Claude
    return llm

In [31]:
def get_llm():
    llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=bedrock_runtime
    # model_kwargs={"max_tokens_to_sample": 1000},
    )
    return llm

In [32]:
def OLD_get_index(): 
    #creates and returns an in-memory vector store to be used in the 
    embeddings = BedrockEmbeddings(
    credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"), #sets the profile name to 
    region_name=os.environ.get("BWB_REGION_NAME"), #sets the region name (if not the defaul
    endpoint_url=os.environ.get("BWB_ENDPOINT_URL"), #sets the endpoint URL (if necessary)
    ) #create a Titan Embeddings client
    pdf_path = "2022-Shareholder-Letter.pdf" #assumes local PDF file with this name
    loader = PyPDFLoader(file_path=pdf_path) #load the pdf file
    text_splitter = RecursiveCharacterTextSplitter( #create a text splitter
    separators=["\n\n", "\n", ".", " "], #split chunks at (1) paragraph, (2) line, (3) sent
    chunk_size=1000, #divide into 1000-character chunks using the separators above
    chunk_overlap=100 #number of characters that can overlap with previous chunk
    )
    index_creator = VectorstoreIndexCreator( #create a vector store factory
    vectorstore_cls=FAISS, #use an in-memory vector store for demo purposes
    embedding=embeddings, #use Titan embeddings
    text_splitter=text_splitter, #use the recursive text splitter
    )
    index_from_loader = index_creator.from_loaders([loader]) #create an vector store index
    return index_from_loader #return the index to be cached by the client app


In [40]:
pdf_path = "polic_docs/Policy_Certificate_Multiple_vehicle.pdf" #assumes local PDF file with this name
loader = PyPDFLoader(file_path=pdf_path) #load the pdf file

documents = loader.load()

print (documents[10].page_content)

Yourpolicyschedule-continued
A1MiniMiniCooperS(122)3h(2007) EffectiveDate27September2023EndDate26September2024
1598Petrol
Automatic3DoorHatchback
Vehiclepolicyholder:Mr.TestTester
Drivingoption: Vehiclepolicyholderonly
-Thisvehiclehascomprehensivecoverbasedon£450excessandupto8000milesayearperyearofSocial,Domestic,Pleasureand
Commutingandrestrictedbusinessuse(refertothevehiclecertificate).
-Thevehiclesareorwillbeownedandregisteredbyeitheryou,yourspouse/civil/domesticpartner,acloserelativeresidingatthesame
address,yourcompanyoristhesubjectofaprivateorpersonalleasingcontract.
-Therearenoadditionaldrivers.
-Vehiclemodifications:nomodifications
-Coverforthisvehicleisbasedon0yearsnoclaimsdiscount(NCD)whichisunprotected.FutureNCDwillbeownedbyMr.Test
Tester.
Yourcover
Thistableshowsyourpolicycoverlimitsintheeventofaclaim.Pleasereadthisalongsideyour
policybooklet.The‘Yourvehicles’sectionwillshowwhichcoveryouhave.Comprehensive
Lossofordamagetoyourvehicle
-Nonmanufacturerfittedaccessorieslimit £1

In [33]:
def get_index(): #creates and returns an in-memory vector store to be used in the appl
    embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_runtime)
    pdf_path = "polic_docs/Policy_Certificate_Multiple_vehicle.pdf" #assumes local PDF file with this name
    loader = PyPDFLoader(file_path=pdf_path) #load the pdf file
    text_splitter = RecursiveCharacterTextSplitter( #create a text splitter
    separators=["\n\n", "\n", ".", " "], #split chunks at (1) paragraph, (2) line, (3) sent
    chunk_size=1000, #divide into 1000-character chunks using the separators above
    chunk_overlap=100 #number of characters that can overlap with previous chunk
    )
    index_creator = VectorstoreIndexCreator( #create a vector store factory
    vectorstore_cls=FAISS, #use an in-memory vector store for demo purposes
    embedding=embeddings, #use Titan embeddings
    text_splitter=text_splitter, #use the recursive text splitter
    )
    index_from_loader = index_creator.from_loaders([loader]) #create an vector store index
    return index_from_loader #return the index to be cached by the client app


In [34]:
def get_memory(): #create memory for this chat session
    memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True)
    return memory
                                            
def get_rag_chat_response(input_text, memory, index): #chat client function
    llm = get_llm()
    conversation_with_retrieval = ConversationalRetrievalChain.from_llm(llm, retriever=index.vectorstore.as_retriever(), memory=memory, verbose=True)
    chat_response = conversation_with_retrieval({"question": input_text}) #pass the user me
    return chat_response['answer']

## TESTING

In [35]:
gpt_index = get_index()
gpt_memory = get_memory()

In [36]:
ques = "Will i be able to recover the vehicle after an accident ?"
get_rag_chat_response(input_text=ques,index=gpt_index, memory=gpt_memory)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

there.
Callimmediatelyfromtheroadside,andwe'll
authoriseamemberofourapprovedrepairer
servicenetworktostartworkandwe'llpaythem
directly.We'llrecoveryourvehicleifit'sunroadworthyin
theeventofanaccident.
Ifyourwindscreenisdamaged,callusrightaway.
Helpisjustonecallaway03450306925
callsmayberecordedand/ormonitoredforourjointprotection-*- Demonstration Powered by OpenText Exstream 09/27/2023, Version 16.6.32 32-bit -*-

1598Petrol
Automatic3DoorHatchback
Vehicle
policyholder:Mr.TestTester
Drivingoption:Vehiclepolicyholderonly
-Thisvehiclehascomprehensivecoverbasedon£450excessandupto8000milesayearperyearofSocial,Domestic,Pleasureand
Commutingandrestrictedbusinessuse(refertothevehiclecertificate).
-Thevehiclesareorwillbeownedand

' Based on the context provided, yes, you will be able to recover the vehicle after an accident if it is unroadworthy. The policy states:\n\n"Call immediately from the roadside, and we\'ll authorise a member of our approved repairer service network to start work and we\'ll pay them directly. We\'ll recovery your vehicle if it\'s unroadworthy in the event of an accident."\n\nThis indicates that if your vehicle is damaged in an accident and becomes unroadworthy (i.e. unsafe or unable to be driven), you can call the insurance company and they will arrange for the vehicle to be recovered. So you should be able to get the vehicle towed or transported after an accident that leaves it unroadworthy.'

In [37]:
ques ="How much is the third party vehicle damage limit ?"
get_rag_chat_response(input_text=ques,index=gpt_index, memory=gpt_memory)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Will i be able to recover the vehicle after an accident ?
Assistant:  Based on the context provided, yes, you will be able to recover the vehicle after an accident if it is unroadworthy. The policy states:

"Call immediately from the roadside, and we'll authorise a member of our approved repairer service network to start work and we'll pay them directly. We'll recovery your vehicle if it's unroadworthy in the event of an accident."

This indicates that if your vehicle is damaged in an accident and becomes unroadworthy (i.e. unsafe or unable to be driven), you can call the insurance company and they will arrange for the vehicle to be recovered. So you should be able to get the vehicle towed or transported after an accident that leaves it unroadworthy.
Follow Up 

' Unfortunately I do not have enough information to determine the third party vehicle damage limit under this policy. The policy schedule provided does not specify limits for third party vehicle damage coverage. It only mentions that the comprehensive policy provides third party liability protection for injury or damage caused to others, but does not give the limit amount. Without knowing the specific third party vehicle damage limit amount, I cannot provide that information.'

In [11]:
ques ="Can i add my motor bike to this policy ?" 
get_rag_chat_response(input_text=ques,index=gpt_index, memory=gpt_memory)

" Unfortunately without having the actual policy details to review, I can't definitively say if a motorcycle could be added to this particular policy. Generally speaking though, most auto insurance policies only cover cars/trucks and do not allow motorcycles to be added. Motorcycle insurance is usually a separate policy. If this were a real scenario, I would recommend contacting the insurance company directly to ask if motorcycle coverage could be added to the existing policy. Since this is hypothetical, I don't have enough information to say for certain one way or the other. The best advice would be to check with the provider when considering adding additional/different vehicles to an existing auto insurance policy."

In [12]:
ques ="How about a van ?"
get_rag_chat_response(input_text=ques,index=gpt_index, memory=gpt_memory)

' Based on the policy details provided, I do not see any indication that you can add a van to this auto insurance policy. The policy seems to only cover the specified vehicles, which are both regular passenger cars (BMW 325i and unspecified model). To add a van, you would likely need to get a separate insurance policy designed for vans or commercial vehicles. The policy states it covers vehicles owned by you, your spouse, close relative, or company, so a van you own could potentially be added, but it would likely require contacting Aviva to modify the policy and re-quote with the van included. Without explicit indication vans are covered, I would not recommend trying to add one to this particular auto policy.'

In [26]:
ques = "what is my current premium and am i covered for windshield break ?"
print (get_rag_chat_response(input_text=ques,index=gpt_index, memory=gpt_memory))

 Based on the policy details provided, the total annual premium for your auto insurance policy is £3,677. This includes £2,089 for vehicle AU53ZWM and £1,588 for vehicle A1. 

The comprehensive cover includes accidental damage, so it should cover windshield replacement if the windshield is damaged. However, the details provided do not explicitly mention windshield coverage. To confirm, you would need to check the full policy terms and conditions. The summary states it provides comprehensive cover, but does not go into specifics on what is included in that.


In [27]:
ques = "Can you check wether im covered for wheel replacement ?"
print (get_rag_chat_response(input_text=ques,index=gpt_index, memory=gpt_memory))


No, the insurance policy provided does not state that it includes coverage for wheel replacement in the case of accidental damage or loss. The policy states it provides comprehensive cover which includes accidental damage, but does not specify coverage for wheels. Without explicit mention of wheel coverage, I would assume it is not included.


### TESTING Template

In [13]:
from langchain import PromptTemplate

custom_prompt = """

Human: I want you to use a document and relevant quotes from the document to answer the question 
. But first access the scentiment of the question and response being calm yet confident. But be as polite as you can. 
{context}
Here are direct quotes from the document that are most relevant to the question 

Please use these to construct an answer to the question as though you were answering the question directly. Ensure that your answer 
is accurate and doesn’t contain any information not directly supported by the document or the quotes.
Also asks for any other question the customer may have as shown below.

Assistant:
"""

CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_prompt)

/opt/conda/lib/python3.10/site-packages/langchain/__init__.py:24: UserWarning: Importing PromptTemplate from langchain root module is no longer supported.
  warnings.warn(


In [14]:
def get_rag_chat_response_template(input_text, memory, index): #chat client function
    llm = get_llm()
    conversation_with_retrieval = ConversationalRetrievalChain.from_llm(llm, retriever=index.vectorstore.as_retriever(), 
                                                                        memory=memory,
                                                                        combine_docs_chain_kwargs={'prompt': CUSTOM_QUESTION_PROMPT})
    
    chat_response = conversation_with_retrieval({"question": input_text}) #pass the user me
    return chat_response['answer']

In [19]:
ques = 'Am i covered for third party liability ?' 
res = get_rag_chat_response_template(input_text=ques,index=gpt_index, memory=gpt_memory)
print_ww(res)

 Based on the provided document, here are the most relevant quotes to answer your question:

"Excesses
Pleaserefertothe‘Yourinsuredvehicles’sectionofthescheduletoseewhatadditionalexcessesapplytoyourvehic
les."

"Comprehensivecover:£1,588.00
Totalvehiclepremium:£1,588.00
Thecomprehensivepolicyprovidesaccidentaldamage,maliciousdamage,fireandtheftcoverforyourvehicleandthi
rdpartyliabilityprotectionforinjuryordamageyoumaycausetoothers."

"Yourinsuredvehicles
AU53ZWMBmw325iSe(192)5e(2003) EffectiveDate27September2023EndDate26September2024
2494Petrol
Manual5DoorEstate"

"Drivingoption: Vehiclepolicyholderonly"

"Coverforthisvehicleisbasedon0yearsnoclaimsdiscount(NCD)whichisunprotected.FutureNCDwillbeownedbyMr.
TestTester."

Based on these quotes, it appears you have comprehensive cover on your BMW 325i SE with a £450
excess. The policy covers accidental damage, malicious damage, fire and theft. It also provides
third party liability protection. The vehicle can only be driven by the policyhol

In [38]:
ques = 'What is my monthly premium and can you tell me the vehicle name in the policy ?' 
res = get_rag_chat_response_template(input_text=ques,index=gpt_index, memory=gpt_memory)
print_ww(res)

 Based on the policy documents provided, here are the relevant details to answer your question:

The comprehensive policy for vehicle AU53ZWM provides accidental damage, malicious damage, fire and
theft cover. It has a £450 excess and is based on up to 15,000 miles per year of use for social,
domestic, pleasure, commuting and restricted business purposes. The policyholder is Mr. Test Tester
and there are no additional drivers.

The comprehensive policy for vehicle A1 provides accidental damage, malicious damage, fire and theft
cover. It has a £450 excess and is based on up to 8,000 miles per year of use for social, domestic,
pleasure, commuting and restricted business purposes. The policyholder is Mr. Test Tester and there
are no additional drivers.

Both vehicles have 0 years no claims discount which is unprotected, and future NCD will be owned by
Mr. Test Tester.

Please let me know if you have any other questions! I'm happy to clarify or provide more details
from the policy document

In [2]:
## Testing commit 123